In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from transformer_lens import utils

from sprint.loading import load_all

/opt/conda/envs/sprint/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model, data, sae = load_all()

Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([215402, 128]), dtype: torch.int64, device: cuda:0
Loading run1 from HuggingFace at 25
{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 2048,
 'dict_mult': 8,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'lr': 0.0001,
 'model_batch_size': 512,
 'num_tokens': 2000000000,
 'seed': 52,
 'seq_len': 128}
Encoder device: cuda:0


In [4]:
prompt_clean = "I can't wait to see you again!"
prompt_corrupted = "I really cant wait to see you again!"

# Check that the tokens line up
tokens_clean = model.to_tokens(prompt_clean).detach().cpu().numpy().flatten()
tokens_corrupted = model.to_tokens(prompt_corrupted).detach().cpu().numpy().flatten()

print("\t".join([model.to_string(t) for t in tokens_clean]))
print("\t".join([model.to_string(t) for t in tokens_corrupted]))

<|BOS|>	I	 can	't	 wait	 to	 see	 you	 again	!
<|BOS|>	I	 really	 cant	 wait	 to	 see	 you	 again	!


In [5]:
# Names
pattern = utils.get_act_name("attn_out", 0)
post = utils.get_act_name("post", 0)

# First, get the corrupt attention pattern
out_corrupt, cache_corrupt = model.run_with_cache(prompt_corrupted, names_filter=[pattern, post])
attn_corrupt = cache_corrupt[pattern]

# Then, get the clean outputs
out_clean, cache_clean = model.run_with_cache(prompt_clean, names_filter=[pattern, post])
attn_clean = cache_clean[pattern]


# Definte the ablation hook
def ablate_attn(activation, hook):
    print(hook.name)
    return attn_corrupt


# Now patch in the corrupt attention pattern and get outputs
with model.hooks(fwd_hooks=[(pattern, ablate_attn)]):
    out_resampled, cache_resampled = model.run_with_cache(prompt_clean, names_filter=[post])
# out_resampled = model.run_with_hooks(prompt_clean, fwd_hooks=[(pattern, ablate_attn), (post, cache_act)])

# Just for fun, the other way around:


def ablate_attn2(activation, hook):
    print(hook.name)
    return attn_clean


# out_resampled2 = model.run_with_hooks(prompt_corrupted, fwd_hooks=[(pattern, ablate_attn2), (post, cache_act2)])
# model.add_hook(pattern, ablate_attn2)
with model.hooks(fwd_hooks=[(pattern, ablate_attn2)]):
    out_resampled2, cache_resampled2 = model.run_with_cache(prompt_corrupted, names_filter=[post])
# model.remove_all_hook_fns()

blocks.0.hook_attn_out
blocks.0.hook_attn_out


In [6]:
FEAT = 10996
TOK = 3

for cache in [cache_corrupt, cache_clean, cache_resampled, cache_resampled2]:
    act = cache[post]
    feat = sae(act[0, TOK])[2].detach().cpu().numpy().flatten()
    print(feat[FEAT])

0.0
1.8125
1.576
0.0


In [7]:
model.hook_dict

{'hook_embed': HookPoint(),
 'hook_pos_embed': HookPoint(),
 'blocks.0.ln1.hook_scale': HookPoint(),
 'blocks.0.ln1.hook_normalized': HookPoint(),
 'blocks.0.ln2.hook_scale': HookPoint(),
 'blocks.0.ln2.hook_normalized': HookPoint(),
 'blocks.0.attn.hook_k': HookPoint(),
 'blocks.0.attn.hook_q': HookPoint(),
 'blocks.0.attn.hook_v': HookPoint(),
 'blocks.0.attn.hook_z': HookPoint(),
 'blocks.0.attn.hook_attn_scores': HookPoint(),
 'blocks.0.attn.hook_pattern': HookPoint(),
 'blocks.0.attn.hook_result': HookPoint(),
 'blocks.0.mlp.hook_pre': HookPoint(),
 'blocks.0.mlp.hook_post': HookPoint(),
 'blocks.0.hook_attn_in': HookPoint(),
 'blocks.0.hook_q_input': HookPoint(),
 'blocks.0.hook_k_input': HookPoint(),
 'blocks.0.hook_v_input': HookPoint(),
 'blocks.0.hook_mlp_in': HookPoint(),
 'blocks.0.hook_attn_out': HookPoint(),
 'blocks.0.hook_mlp_out': HookPoint(),
 'blocks.0.hook_resid_pre': HookPoint(),
 'blocks.0.hook_resid_mid': HookPoint(),
 'blocks.0.hook_resid_post': HookPoint(),
 'l